In [1]:
# Setup Cell 1 - Imports
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import gzip
import requests
from pathlib import Path
import sqlite3
import json
import re
import warnings
from typing import Dict, List, Optional, Union
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import time

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Setup Cell 2 - Configure display and warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('default')  # Use default if seaborn issues
sns.set_palette("husl")

print("✅ Display options configured")

✅ Display options configured


In [3]:
# Setup Cell 3 - Project paths
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
DISCOGS_DIR = DATA_DIR / 'discogs'
PROCESSED_DIR = DATA_DIR / 'processed'
RAW_DIR = DATA_DIR / 'raw'

# Create directories
DISCOGS_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Discogs directory: {DISCOGS_DIR}")
print(f"📁 Processed directory: {PROCESSED_DIR}")
print("✅ Directory structure created")

📁 Project root: /Users/richpointofview/smart-vinyl-catalog
📁 Data directory: /Users/richpointofview/smart-vinyl-catalog/data
📁 Discogs directory: /Users/richpointofview/smart-vinyl-catalog/data/discogs
📁 Processed directory: /Users/richpointofview/smart-vinyl-catalog/data/processed
✅ Directory structure created


In [4]:
# Cell 1 - Load Existing Catalog
# Load the FMA-integrated catalog from the previous notebook

# Load existing FMA catalog
fma_catalog_path = PROCESSED_DIR / 'fma_integrated.csv'

if fma_catalog_path.exists():
    existing_catalog = pd.read_csv(fma_catalog_path)
    print(f"✅ Loaded existing catalog: {len(existing_catalog):,} tracks")
    
    # Check if source column exists, if not add it
    if 'source' not in existing_catalog.columns:
        existing_catalog['source'] = 'fma_data'  # Default source for FMA data
        print("📝 Added 'source' column (set to 'fma_data')")
    
    print(f"📊 Sources: {existing_catalog['source'].value_counts().to_dict()}")
    
    # Display basic stats
    print(f"\n📈 Catalog Statistics:")
    print(f"   🎵 Unique genres: {existing_catalog['genre'].nunique() if 'genre' in existing_catalog.columns else 'N/A'}")
    print(f"   🎤 Unique artists: {existing_catalog['artist'].nunique() if 'artist' in existing_catalog.columns else 'N/A'}")
    print(f"   ⭐ Average rating: {existing_catalog['rating'].mean():.2f}" if 'rating' in existing_catalog.columns else "   ⭐ No ratings available")
    print(f"   📅 Year range: {existing_catalog['year'].min():.0f} - {existing_catalog['year'].max():.0f}" if 'year' in existing_catalog.columns else "   📅 No year data")
    
else:
    print("⚠️  FMA catalog not found. Creating sample catalog for demonstration.")
    # Create sample data that matches FMA structure
    existing_catalog = pd.DataFrame({
        'track_id': range(100),
        'title': [f'Sample Track {i+1}' for i in range(100)],
        'artist': ['Various Artists'] * 100,
        'genre': np.random.choice(['Electronic', 'Rock', 'Jazz', 'Folk'], 100),
        'rating': np.random.normal(3.5, 0.8, 100),
        'year': np.random.randint(1990, 2024, 100),
        'duration': np.random.randint(120, 400, 100),
        'plays': np.random.randint(100, 10000, 100),
        'favorites': np.random.randint(10, 1000, 100),
        'source': 'sample_data'
    })
    print(f"📝 Created sample catalog: {len(existing_catalog)} tracks")

# Display sample
print(f"\n🔍 Existing catalog sample:")
display(existing_catalog.head())

print(f"\n📋 Column info:")
print(f"   Columns: {list(existing_catalog.columns)}")
print(f"   Shape: {existing_catalog.shape}")
print(f"   Memory usage: {existing_catalog.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")

✅ Loaded existing catalog: 5,000 tracks
📝 Added 'source' column (set to 'fma_data')
📊 Sources: {'fma_data': 5000}

📈 Catalog Statistics:
   🎵 Unique genres: 16
   🎤 Unique artists: 3139
   ⭐ Average rating: 2.00
   📅 Year range: 1971 - 2017

🔍 Existing catalog sample:


,release_id,title,artist,album,genre,year,duration,tags,plays,favorites,license,popularity_score,rating,label,country,review_text,review_source,source
0,129624,Tentacle Hentaij - Core Noisetone,Ralph Brown,Spettro Records Volume 2 - Decline,Experimental,2015.0,265,[],0,1,Attribution-Noncommercial-Share Alike 3.0 Unit...,0.000202,2.0,FMA,US,Creative Commons track by Ralph Brown. Genre: ...,FMA_Metadata,fma_data
1,24769,Goldfish,Aoiroooasamusi,Root Of Sorrow,NaN,2010.0,308,[],0,17,Attribution-Noncommercial-Share Alike 3.0 Unit...,0.003441,2.0,FMA,US,Creative Commons track by Aoiroooasamusi. Genr...,FMA_Metadata,fma_data
2,14537,The Monk Said,dmyra,Love in the Air,NaN,2009.0,388,"['clinical archives', 'folk', 'rock', 'psych-f...",0,4,Attribution-NonCommercial-NoDerivatives (aka M...,0.000810,2.0,FMA,US,Creative Commons track by dmyra. Genre: nan. [...,FMA_Metadata,fma_data
3,36538,Fac ut animae,The Tudor Consort,Stabat Mater - Domenico Scarlatti,Classical,2010.0,129,['new zealand'],0,11,Attribution 3.0 International,0.002227,2.0,FMA,US,Creative Commons track by The Tudor Consort. G...,FMA_Metadata,fma_data
4,25193,Bartlett Bridge,Vitamin-D,Live at WFMU on Irene Trudel's show 2/8/10,Folk,2010.0,173,[],0,0,Attribution-Noncommercial-No Derivative Works ...,0.000000,2.0,FMA,US,Creative Commons track by Vitamin-D. Genre: Fo...,FMA_Metadata,fma_data



📋 Column info:
   Columns: ['release_id', 'title', 'artist', 'album', 'genre', 'year', 'duration', 'tags', 'plays', 'favorites', 'license', 'popularity_score', 'rating', 'label', 'country', 'review_text', 'review_source', 'source']
   Shape: (5000, 18)
   Memory usage: 4.0 MB


In [5]:
# Cell 2 - Kaggle API Setup and Data Download Preparation

def check_kaggle_setup():
    """Check if Kaggle API is properly configured"""
    try:
        import kaggle
        # Try to authenticate
        kaggle.api.authenticate()
        print("✅ Kaggle API is configured and authenticated")
        return True
    except ImportError:
        print("❌ Kaggle package not installed")
        print("💡 Install with: pip install kaggle")
        return False
    except Exception as e:
        print(f"❌ Kaggle API error: {e}")
        print("\n🔧 To set up Kaggle API:")
        print("1. Install: pip install kaggle")
        print("2. Go to kaggle.com → Account → Create New API Token")
        print("3. Download kaggle.json file")
        print("4. Place kaggle.json in ~/.kaggle/ directory")
        print("5. Run: chmod 600 ~/.kaggle/kaggle.json (on Mac/Linux)")
        return False

def check_discogs_data_exists():
    """Check if Discogs data already exists in our directory"""
    print(f"\n📁 Checking existing files in {DISCOGS_DIR}...")
    
    # Look for common Discogs file patterns
    patterns = ['*discogs*', '*releases*', '*artists*', '*labels*', '*.xml*', '*.csv']
    existing_files = []
    
    for pattern in patterns:
        files = list(DISCOGS_DIR.glob(pattern))
        existing_files.extend(files)
    
    # Remove duplicates and sort
    existing_files = sorted(set(existing_files))
    
    if existing_files:
        print(f"📄 Found {len(existing_files)} existing files:")
        for f in existing_files[:10]:  # Show first 10
            size_mb = f.stat().st_size / 1024 / 1024
            print(f"   📄 {f.name} ({size_mb:.1f} MB)")
        if len(existing_files) > 10:
            print(f"   ... and {len(existing_files) - 10} more files")
        return True
    else:
        print("📭 No existing Discogs files found")
        return False

# Check Kaggle setup
print("🔍 Checking Kaggle API setup...")
kaggle_ready = check_kaggle_setup()

# Check for existing data
data_exists = check_discogs_data_exists()

# Summary
print(f"\n📋 Status Summary:")
print(f"   Kaggle API Ready: {'✅' if kaggle_ready else '❌'}")
print(f"   Existing Data: {'✅' if data_exists else '📭'}")

if not kaggle_ready and not data_exists:
    print(f"\n⚠️  Next steps:")
    print(f"   1. Set up Kaggle API (see instructions above)")
    print(f"   2. Or manually download Discogs data")
elif kaggle_ready and not data_exists:
    print(f"\n✅ Ready to download Discogs data in next cell!")
elif data_exists:
    print(f"\n✅ Can proceed with existing data or download fresh data!")
else:
    print(f"\n✅ Ready to proceed!")

🔍 Checking Kaggle API setup...
❌ Kaggle API error: Could not find kaggle.json. Make sure it's located in /Users/richpointofview/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

🔧 To set up Kaggle API:
1. Install: pip install kaggle
2. Go to kaggle.com → Account → Create New API Token
3. Download kaggle.json file
4. Place kaggle.json in ~/.kaggle/ directory
5. Run: chmod 600 ~/.kaggle/kaggle.json (on Mac/Linux)

📁 Checking existing files in /Users/richpointofview/smart-vinyl-catalog/data/discogs...
📭 No existing Discogs files found

📋 Status Summary:
   Kaggle API Ready: ❌
   Existing Data: 📭

⚠️  Next steps:
   1. Set up Kaggle API (see instructions above)
   2. Or manually download Discogs data


In [7]:
# Cell 3 - Download Discogs Data (Multiple Methods)

def download_discogs_kaggle(dataset_name: str = "ofurkancoban/discogs-data-dumps-april-2025"):
    """Download Discogs dataset from Kaggle"""
    try:
        import kaggle
        kaggle.api.authenticate()
        
        print(f"📥 Downloading {dataset_name}...")
        kaggle.api.dataset_download_files(
            dataset_name,
            path=str(DISCOGS_DIR),
            unzip=True
        )
        
        print("✅ Download complete!")
        return True
        
    except Exception as e:
        print(f"❌ Kaggle download failed: {e}")
        return False

def download_discogs_manual():
    """Provide manual download instructions"""
    print("🔗 Manual Download Options:")
    print("\n📋 Option 1 - Kaggle (Recommended):")
    print("   1. Go to: https://www.kaggle.com/datasets/ofurkancoban/discogs-data-dumps-april-2025")
    print("   2. Click 'Download' button")
    print(f"   3. Extract files to: {DISCOGS_DIR}")
    
    print("\n📋 Option 2 - Official Discogs (Large files):")
    print("   1. Go to: https://discogs-data-dumps.s3.us-west-2.amazonaws.com/index.html")
    print("   2. Download recent releases.xml.gz file")
    print(f"   3. Save to: {DISCOGS_DIR}")
    
    print(f"\n📁 Target directory: {DISCOGS_DIR}")
    
def create_sample_discogs_data():
    """Create sample Discogs-style data for testing"""
    print("🎯 Creating sample Discogs data for testing...")
    
    # Create sample releases data
    sample_releases = []
    
    # Sample classic vinyl releases
    classic_data = [
        {"title": "Kind of Blue", "artist": "Miles Davis", "label": "Columbia", "year": 1959, "genre": "Jazz", "country": "US"},
        {"title": "Pet Sounds", "artist": "The Beach Boys", "label": "Capitol", "year": 1966, "genre": "Pop", "country": "US"},
        {"title": "Sgt. Pepper's Lonely Hearts Club Band", "artist": "The Beatles", "label": "Parlophone", "year": 1967, "genre": "Rock", "country": "UK"},
        {"title": "What's Going On", "artist": "Marvin Gaye", "label": "Tamla", "year": 1971, "genre": "Soul", "country": "US"},
        {"title": "The Dark Side of the Moon", "artist": "Pink Floyd", "label": "Harvest", "year": 1973, "genre": "Progressive Rock", "country": "UK"},
        {"title": "Songs in the Key of Life", "artist": "Stevie Wonder", "label": "Tamla", "year": 1976, "genre": "Soul", "country": "US"},
        {"title": "Rumours", "artist": "Fleetwood Mac", "label": "Warner Bros.", "year": 1977, "genre": "Rock", "country": "US"},
        {"title": "Unknown Pleasures", "artist": "Joy Division", "label": "Factory", "year": 1979, "genre": "Post-Punk", "country": "UK"},
        {"title": "London Calling", "artist": "The Clash", "label": "CBS", "year": 1979, "genre": "Punk", "country": "UK"},
        {"title": "Purple Rain", "artist": "Prince", "label": "Warner Bros.", "year": 1984, "genre": "Pop", "country": "US"}
    ]
    
    # Expand sample data
    for i in range(1000):
        base_record = classic_data[i % len(classic_data)]
        
        sample_releases.append({
            'discogs_id': f'discogs_{i+1}',
            'title': base_record['title'] if i < 50 else f"{base_record['title']} (Reissue {i//50})",
            'artist': base_record['artist'],
            'label': base_record['label'],
            'year': base_record['year'] + (i // 100),  # Vary years
            'genre': base_record['genre'],
            'country': base_record['country'],
            'format': np.random.choice(['LP', '12"', '7"'], p=[0.7, 0.2, 0.1]),
            'rating': np.random.normal(4.1, 0.6),
            'catalog_number': f"CAT-{i+1000}",
            'status': 'Accepted',
            'source': 'discogs_sample'
        })
    
    # Create DataFrame
    sample_df = pd.DataFrame(sample_releases)
    sample_df['rating'] = sample_df['rating'].clip(1, 5).round(1)
    
    # Save to CSV for easier handling
    sample_path = DISCOGS_DIR / 'sample_discogs_releases.csv'
    sample_df.to_csv(sample_path, index=False)
    
    print(f"✅ Created {len(sample_df)} sample Discogs releases")
    print(f"📄 Saved to: {sample_path}")
    return sample_df

# Main execution
print("🎯 Choose your download method:")
print("1. 🔑 Set up Kaggle API first (recommended)")
print("2. 📥 Try Kaggle download anyway")
print("3. 📋 Manual download instructions")
print("4. 🎯 Create sample data for testing")

choice = input("\nEnter choice (1-4): ").strip()

if choice == '1':
    print("\n🔑 Kaggle API Setup Instructions:")
    print("1. Go to kaggle.com and log in")
    print("2. Go to Account > API > Create New API Token")
    print("3. Download kaggle.json file")
    print("4. Run these commands in terminal:")
    print(f"   mkdir -p ~/.kaggle")
    print(f"   mv ~/Downloads/kaggle.json ~/.kaggle/")
    print(f"   chmod 600 ~/.kaggle/kaggle.json")
    print("5. Come back and run this cell again with option 2")
    
elif choice == '2':
    success = download_discogs_kaggle()
    if success:
        # Check what we downloaded
        files = list(DISCOGS_DIR.glob('*'))
        print(f"\n📁 Downloaded files ({len(files)}):")
        for f in files[:10]:
            print(f"   📄 {f.name}")
    
elif choice == '3':
    download_discogs_manual()
    
elif choice == '4':
    sample_data = create_sample_discogs_data()
    print(f"\n📊 Sample data preview:")
    print(sample_data[['title', 'artist', 'label', 'year', 'genre', 'rating']].head())
    
else:
    print("❌ Invalid choice. Please run the cell again and choose 1-4.")


🎯 Choose your download method:
1. 🔑 Set up Kaggle API first (recommended)
2. 📥 Try Kaggle download anyway
3. 📋 Manual download instructions
4. 🎯 Create sample data for testing
🎯 Creating sample Discogs data for testing...
✅ Created 1000 sample Discogs releases
📄 Saved to: /Users/richpointofview/smart-vinyl-catalog/data/discogs/sample_discogs_releases.csv

📊 Sample data preview:
                                   title          artist       label  year  \
0                           Kind of Blue     Miles Davis    Columbia  1959   
1                             Pet Sounds  The Beach Boys     Capitol  1966   
2  Sgt. Pepper's Lonely Hearts Club Band     The Beatles  Parlophone  1967   
3                        What's Going On     Marvin Gaye       Tamla  1971   
4              The Dark Side of the Moon      Pink Floyd     Harvest  1973   

              genre  rating  
0              Jazz     3.5  
1               Pop     5.0  
2              Rock     3.8  
3              Soul     3.5  


In [8]:
# Cell 4 - Data Integration: Merge FMA and Discogs Data

def load_discogs_sample():
    """Load the sample Discogs data we created"""
    discogs_path = DISCOGS_DIR / 'sample_discogs_releases.csv'
    
    if discogs_path.exists():
        discogs_df = pd.read_csv(discogs_path)
        print(f"✅ Loaded Discogs sample: {len(discogs_df):,} releases")
        return discogs_df
    else:
        print("❌ Discogs sample file not found!")
        return None

def standardize_discogs_columns(discogs_df):
    """Standardize Discogs columns to match FMA catalog structure"""
    print("🔄 Standardizing Discogs data structure...")
    
    standardized = discogs_df.copy()
    
    # Add missing columns that exist in FMA catalog
    required_columns = {
        'duration': lambda: np.random.randint(180, 2400, len(standardized)),  # 3-40 minutes
        'plays': lambda: np.random.randint(1, 1000, len(standardized)),
        'favorites': lambda: np.random.randint(1, 200, len(standardized)),
        'review_text': lambda: standardized.apply(
            lambda x: f"Classic {x['genre']} album by {x['artist']} on {x['label']} Records from {x['year']}", 
            axis=1
        )
    }
    
    for col, generator in required_columns.items():
        if col not in standardized.columns:
            standardized[col] = generator()
    
    # Ensure consistent column names and types
    column_mapping = {
        'discogs_id': 'release_id'
    }
    
    for old_col, new_col in column_mapping.items():
        if old_col in standardized.columns:
            standardized[new_col] = standardized[old_col]
    
    # Ensure data types match
    if 'year' in standardized.columns:
        standardized['year'] = pd.to_numeric(standardized['year'], errors='coerce')
    if 'rating' in standardized.columns:
        standardized['rating'] = pd.to_numeric(standardized['rating'], errors='coerce')
    
    print(f"✅ Standardized {len(standardized)} Discogs records")
    return standardized

def merge_catalogs(fma_catalog, discogs_catalog):
    """Merge FMA and Discogs catalogs into unified database"""
    print("🔄 Merging catalogs...")
    
    # Ensure both catalogs have consistent columns
    all_columns = set(fma_catalog.columns) | set(discogs_catalog.columns)
    
    # Add missing columns to each catalog
    for col in all_columns:
        if col not in fma_catalog.columns:
            if col in ['label', 'catalog_number', 'country', 'format']:
                fma_catalog[col] = 'Unknown'
            elif col in ['duration', 'plays', 'favorites']:
                fma_catalog[col] = np.random.randint(100, 500, len(fma_catalog))
            else:
                fma_catalog[col] = None
    
    for col in all_columns:
        if col not in discogs_catalog.columns:
            if col in ['track_id']:
                discogs_catalog[col] = range(len(discogs_catalog))
            else:
                discogs_catalog[col] = None
    
    # Combine the catalogs
    combined = pd.concat([fma_catalog, discogs_catalog], ignore_index=True)
    
    # Clean up the combined data
    combined = combined.fillna('Unknown')
    
    print(f"✅ Combined catalog created:")
    print(f"   📊 Total records: {len(combined):,}")
    print(f"   📊 FMA records: {len(fma_catalog):,}")
    print(f"   📊 Discogs records: {len(discogs_catalog):,}")
    
    return combined

def analyze_combined_catalog(combined_catalog):
    """Analyze the combined catalog"""
    print("\n📊 Combined Catalog Analysis:")
    
    # Source breakdown
    if 'source' in combined_catalog.columns:
        source_counts = combined_catalog['source'].value_counts()
        print(f"\n📈 Records by Source:")
        for source, count in source_counts.items():
            print(f"   {source}: {count:,}")
    
    # Genre distribution
    if 'genre' in combined_catalog.columns:
        genre_counts = combined_catalog['genre'].value_counts().head(10)
        print(f"\n🎵 Top 10 Genres:")
        for genre, count in genre_counts.items():
            print(f"   {genre}: {count:,}")
    
    # Year range
    if 'year' in combined_catalog.columns:
        year_data = pd.to_numeric(combined_catalog['year'], errors='coerce').dropna()
        if len(year_data) > 0:
            print(f"\n📅 Year Range: {int(year_data.min())} - {int(year_data.max())}")
    
    # Rating statistics
    if 'rating' in combined_catalog.columns:
        rating_data = pd.to_numeric(combined_catalog['rating'], errors='coerce').dropna()
        if len(rating_data) > 0:
            print(f"\n⭐ Rating Statistics:")
            print(f"   Average: {rating_data.mean():.2f}")
            print(f"   Median: {rating_data.median():.2f}")
            print(f"   Range: {rating_data.min():.1f} - {rating_data.max():.1f}")
    
    # Memory usage
    memory_mb = combined_catalog.memory_usage(deep=True).sum() / 1024 / 1024
    print(f"\n💾 Memory Usage: {memory_mb:.1f} MB")
    
    return combined_catalog

# Main execution
print("🚀 Starting data integration process...")

# Step 1: Load Discogs sample data
discogs_sample = load_discogs_sample()

if discogs_sample is not None:
    # Step 2: Standardize Discogs data structure
    discogs_standardized = standardize_discogs_columns(discogs_sample)
    
    # Step 3: Merge with existing FMA catalog
    combined_catalog = merge_catalogs(existing_catalog, discogs_standardized)
    
    # Step 4: Analyze the combined catalog
    final_catalog = analyze_combined_catalog(combined_catalog)
    
    # Step 5: Save the combined catalog
    output_path = PROCESSED_DIR / 'combined_catalog_with_discogs.csv'
    final_catalog.to_csv(output_path, index=False)
    
    print(f"\n💾 Combined catalog saved to: {output_path}")
    print(f"📄 File size: {output_path.stat().st_size / 1024 / 1024:.1f} MB")
    
    # Step 6: Show sample of combined data
    print(f"\n🔍 Sample of combined catalog:")
    sample_cols = ['title', 'artist', 'genre', 'year', 'rating', 'source']
    available_cols = [col for col in sample_cols if col in final_catalog.columns]
    display(final_catalog[available_cols].head(10))
    
else:
    print("❌ Cannot proceed without Discogs data. Please run Cell 3 first.")

🚀 Starting data integration process...
✅ Loaded Discogs sample: 1,000 releases
🔄 Standardizing Discogs data structure...
✅ Standardized 1000 Discogs records
🔄 Merging catalogs...
✅ Combined catalog created:
   📊 Total records: 6,000
   📊 FMA records: 5,000
   📊 Discogs records: 1,000

📊 Combined Catalog Analysis:

📈 Records by Source:
   fma_data: 5,000
   discogs_sample: 1,000

🎵 Top 10 Genres:
   Unknown: 2,714
   Rock: 837
   Experimental: 486
   Electronic: 448
   Pop: 307
   Soul: 200
   Hip-Hop: 180
   Folk: 130
   Jazz: 116
   Post-Punk: 100

📅 Year Range: 1959 - 2017

⭐ Rating Statistics:
   Average: 2.35
   Median: 2.00
   Range: 2.0 - 5.0

💾 Memory Usage: 6.5 MB

💾 Combined catalog saved to: /Users/richpointofview/smart-vinyl-catalog/data/processed/combined_catalog_with_discogs.csv
📄 File size: 1.8 MB

🔍 Sample of combined catalog:


,title,artist,genre,year,rating,source
0,Tentacle Hentaij - Core Noisetone,Ralph Brown,Experimental,2015.0,2.0,fma_data
1,Goldfish,Aoiroooasamusi,Unknown,2010.0,2.0,fma_data
2,The Monk Said,dmyra,Unknown,2009.0,2.0,fma_data
3,Fac ut animae,The Tudor Consort,Classical,2010.0,2.0,fma_data
4,Bartlett Bridge,Vitamin-D,Folk,2010.0,2.0,fma_data
5,Midnight beat (goes on),Itsensäsyöjät,Electronic,Unknown,2.0,fma_data
6,Volkslied,Uncle Woody Sullender and Seamus Cater,Unknown,2010.0,2.0,fma_data
7,160 friki break,Bauchamp,Unknown,Unknown,2.0,fma_data
8,The Darkest Streets,Rest You Sleeping Giant,Unknown,Unknown,2.0,fma_data
9,Dr Mudd,half cocked,Rock,Unknown,2.0,fma_data


In [9]:
# Cell 5 - Enhanced Search and Recommendations with Combined Catalog

class EnhancedMusicSearchEngine:
    """Advanced search engine for the combined FMA + Discogs catalog"""
    
    def __init__(self, catalog_df):
        self.catalog = catalog_df.copy()
        self.catalog = self.catalog.fillna('')  # Handle NaN values
        
        # Create search indices for faster lookups
        self._create_search_indices()
        
    def _create_search_indices(self):
        """Create search indices for common queries"""
        print("Building search indices...")
        
        # Create searchable text for each record
        self.catalog['searchable_text'] = (
            self.catalog['title'].astype(str) + ' ' +
            self.catalog['artist'].astype(str) + ' ' +
            self.catalog['genre'].astype(str) + ' ' +
            self.catalog['label'].astype(str, errors='ignore').fillna('')
        ).str.lower()
        
        print(f"Search indices built for {len(self.catalog)} records")
    
    def search_by_keywords(self, query, max_results=10):
        """Search by keywords across title, artist, genre, label"""
        query_lower = str(query).lower()
        
        # Find matches in searchable text
        matches = self.catalog[
            self.catalog['searchable_text'].str.contains(query_lower, case=False, na=False)
        ].copy()
        
        if len(matches) > 0:
            # Calculate relevance score based on exact matches
            matches['relevance'] = matches['searchable_text'].apply(
                lambda x: x.count(query_lower)
            )
            
            # Sort by relevance and rating
            matches = matches.sort_values(['relevance', 'rating'], ascending=[False, False])
            
            return matches.head(max_results)
        
        return pd.DataFrame()
    
    def search_by_genre(self, genre, min_rating=2.0, max_results=15):
        """Search for high-rated music in specific genre"""
        matches = self.catalog[
            (self.catalog['genre'].str.contains(genre, case=False, na=False)) &
            (pd.to_numeric(self.catalog['rating'], errors='coerce') >= min_rating)
        ].copy()
        
        if len(matches) > 0:
            matches = matches.sort_values('rating', ascending=False)
            return matches.head(max_results)
        
        return pd.DataFrame()
    
    def search_cross_source(self, query, compare_sources=True):
        """Search across both FMA and Discogs data sources"""
        results = self.search_by_keywords(query, max_results=20)
        
        if len(results) > 0 and compare_sources:
            # Group by source
            fma_results = results[results['source'] == 'fma_data']
            discogs_results = results[results['source'] == 'discogs_sample']
            
            print(f"Search results for '{query}':")
            print(f"  FMA matches: {len(fma_results)}")
            print(f"  Discogs matches: {len(discogs_results)}")
            
            return {
                'all_results': results,
                'fma_results': fma_results,
                'discogs_results': discogs_results
            }
        
        return {'all_results': results}

class EnhancedRecommendationEngine:
    """Advanced recommendation engine using combined catalog"""
    
    def __init__(self, catalog_df):
        self.catalog = catalog_df.copy()
        self.catalog = self.catalog.fillna('')
        
    def recommend_by_genre_cross_source(self, target_genre, max_results=10):
        """Get recommendations across both data sources for a genre"""
        genre_matches = self.catalog[
            self.catalog['genre'].str.contains(target_genre, case=False, na=False)
        ].copy()
        
        if len(genre_matches) == 0:
            return []
        
        # Sort by rating and get diverse recommendations
        recommendations = genre_matches.sort_values('rating', ascending=False).head(max_results)
        
        results = []
        for _, track in recommendations.iterrows():
            results.append({
                'title': track['title'],
                'artist': track['artist'],
                'genre': track['genre'],
                'year': track['year'],
                'rating': track['rating'],
                'source': track['source'],
                'reason': f"High-rated {target_genre} from {track['source'].replace('_', ' ').title()}"
            })
        
        return results
    
    def discover_classics_vs_underground(self, genre='Rock', max_each=5):
        """Compare classic (Discogs) vs underground (FMA) in same genre"""
        genre_matches = self.catalog[
            self.catalog['genre'].str.contains(genre, case=False, na=False)
        ].copy()
        
        if len(genre_matches) == 0:
            return {'classics': [], 'underground': []}
        
        # Split by source
        classics = genre_matches[
            genre_matches['source'] == 'discogs_sample'
        ].sort_values('rating', ascending=False).head(max_each)
        
        underground = genre_matches[
            genre_matches['source'] == 'fma_data'
        ].sort_values('rating', ascending=False).head(max_each)
        
        return {
            'classics': classics[['title', 'artist', 'year', 'rating']].to_dict('records'),
            'underground': underground[['title', 'artist', 'year', 'rating']].to_dict('records')
        }

# Initialize engines with combined catalog
print("Initializing enhanced search and recommendation engines...")

search_engine = EnhancedMusicSearchEngine(final_catalog)
recommendation_engine = EnhancedRecommendationEngine(final_catalog)

print("Engines initialized successfully!")

# Demonstration searches
print("\n" + "="*50)
print("DEMONSTRATION: Enhanced Search Capabilities")
print("="*50)

# Demo 1: Cross-source keyword search
print("\n1. Cross-source search for 'jazz':")
jazz_results = search_engine.search_cross_source('jazz')
if len(jazz_results['all_results']) > 0:
    for _, track in jazz_results['all_results'].head(5).iterrows():
        print(f"   {track['title']} by {track['artist']} ({track['source']}) - Rating: {track['rating']}")

# Demo 2: Genre-specific search
print("\n2. High-rated Electronic music:")
electronic_results = search_engine.search_by_genre('Electronic', min_rating=3.0)
if len(electronic_results) > 0:
    for _, track in electronic_results.head(5).iterrows():
        print(f"   {track['title']} by {track['artist']} - Rating: {track['rating']} ({track['source']})")

# Demo 3: Cross-source recommendations
print("\n3. Rock recommendations across sources:")
rock_recs = recommendation_engine.recommend_by_genre_cross_source('Rock', max_results=6)
for rec in rock_recs:
    print(f"   {rec['title']} by {rec['artist']} - {rec['reason']}")

# Demo 4: Classics vs Underground comparison
print("\n4. Jazz: Classics vs Underground comparison:")
jazz_comparison = recommendation_engine.discover_classics_vs_underground('Jazz')
print("   CLASSICS (Discogs):")
for track in jazz_comparison['classics'][:3]:
    print(f"     {track['title']} by {track['artist']} ({track['year']}) - {track['rating']}")
print("   UNDERGROUND (FMA):")
for track in jazz_comparison['underground'][:3]:
    print(f"     {track['title']} by {track['artist']} ({track['year']}) - {track['rating']}")

print("\n" + "="*50)
print("Search and recommendation engines ready for use!")
print("="*50)

Initializing enhanced search and recommendation engines...
Building search indices...
Search indices built for 6000 records
Engines initialized successfully!

DEMONSTRATION: Enhanced Search Capabilities

1. Cross-source search for 'jazz':
Search results for 'jazz':
  FMA matches: 0
  Discogs matches: 20
   Kind of Blue by Miles Davis (discogs_sample) - Rating: 5.0
   Kind of Blue (Reissue 5) by Miles Davis (discogs_sample) - Rating: 5.0
   Kind of Blue (Reissue 6) by Miles Davis (discogs_sample) - Rating: 5.0
   Kind of Blue (Reissue 7) by Miles Davis (discogs_sample) - Rating: 5.0
   Kind of Blue (Reissue 9) by Miles Davis (discogs_sample) - Rating: 5.0

2. High-rated Electronic music:

3. Rock recommendations across sources:
   Sgt. Pepper's Lonely Hearts Club Band (Reissue 4) by The Beatles - High-rated Rock from Discogs Sample
   Rumours (Reissue 14) by Fleetwood Mac - High-rated Rock from Discogs Sample
   The Dark Side of the Moon (Reissue 16) by Pink Floyd - High-rated Rock from

In [ ]:
# Cell 6 - Visualization and Analysis Dashboard (Complete Fixed Version)

def create_source_comparison_viz(catalog_df):
    """Create visualizations comparing FMA vs Discogs data"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('FMA vs Discogs Data Comparison', fontsize=16, fontweight='bold')
    
    # 1. Source distribution
    source_counts = catalog_df['source'].value_counts()
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'][:len(source_counts)]
    axes[0, 0].pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%', 
                   colors=colors)
    axes[0, 0].set_title('Data Source Distribution')
    
    # 2. Rating distribution by source
    fma_ratings = pd.to_numeric(catalog_df[catalog_df['source'] == 'fma_data']['rating'], errors='coerce').dropna()
    discogs_ratings = pd.to_numeric(catalog_df[catalog_df['source'] == 'discogs_sample']['rating'], errors='coerce').dropna()
    
    if len(fma_ratings) > 0:
        axes[0, 1].hist(fma_ratings, bins=20, alpha=0.7, label='FMA', color='#ff9999')
    if len(discogs_ratings) > 0:
        axes[0, 1].hist(discogs_ratings, bins=20, alpha=0.7, label='Discogs', color='#66b3ff')
    axes[0, 1].set_xlabel('Rating')
    axes[0, 1].set_ylabel('Count')
    axes[0, 1].set_title('Rating Distribution by Source')
    axes[0, 1].legend()
    
    # 3. Genre distribution
    genre_counts = catalog_df['genre'].value_counts().head(10)
    y_pos = range(len(genre_counts))
    axes[1, 0].barh(y_pos, genre_counts.values, color='#99ff99')
    axes[1, 0].set_yticks(y_pos)
    axes[1, 0].set_yticklabels(genre_counts.index)
    axes[1, 0].set_xlabel('Count')
    axes[1, 0].set_title('Top 10 Genres')
    
    # 4. Year distribution by decade
    years = pd.to_numeric(catalog_df['year'], errors='coerce').dropna()
    if len(years) > 0:
        min_year = int(years.min())
        max_year = int(years.max())
        decade_start = (min_year // 10) * 10
        decade_end = ((max_year // 10) + 1) * 10
        decade_bins = range(decade_start, decade_end, 10)
        
        axes[1, 1].hist(years, bins=decade_bins, color='#ffcc99', edgecolor='black', alpha=0.7)
        axes[1, 1].set_xlabel('Year')
        axes[1, 1].set_ylabel('Count')
        axes[1, 1].set_title('Release Years Distribution')
        axes[1, 1].tick_params(axis='x', rotation=45)
    else:
        axes[1, 1].text(0.5, 0.5, 'No year data available', ha='center', va='center')
        axes[1, 1].set_title('Release Years Distribution')
    
    plt.tight_layout()
    plt.show()

def analyze_catalog_statistics(catalog_df):
    """Generate comprehensive statistics about the catalog"""
    print("📊 COMPREHENSIVE CATALOG STATISTICS")
    print("=" * 50)
    
    # Basic statistics
    total_records = len(catalog_df)
    print(f"Total Records: {total_records:,}")
    
    # Source breakdown
    source_counts = catalog_df['source'].value_counts()
    print(f"\nSource Breakdown:")
    for source, count in source_counts.items():
        percentage = (count / total_records) * 100
        print(f"  {source.replace('_', ' ').title()}: {count:,} ({percentage:.1f}%)")
    
    # Genre analysis
    genre_counts = catalog_df['genre'].value_counts()
    print(f"\nGenre Diversity:")
    print(f"  Unique genres: {len(genre_counts)}")
    if len(genre_counts) > 0:
        print(f"  Most common: {genre_counts.index[0]} ({genre_counts.iloc[0]:,} tracks)")
        print(f"  Top 5 genres: {', '.join(genre_counts.head(5).index.tolist())}")
    
    # Rating analysis
    ratings = pd.to_numeric(catalog_df['rating'], errors='coerce').dropna()
    if len(ratings) > 0:
        print(f"\nRating Statistics:")
        print(f"  Records with ratings: {len(ratings):,}")
        print(f"  Average rating: {ratings.mean():.2f}")
        print(f"  Median rating: {ratings.median():.2f}")
        print(f"  Highest rated: {ratings.max():.1f}")
        print(f"  Lowest rated: {ratings.min():.1f}")
        print(f"  Standard deviation: {ratings.std():.2f}")
    else:
        print(f"\nRating Statistics: No numeric ratings found")
    
    # Year analysis
    years = pd.to_numeric(catalog_df['year'], errors='coerce').dropna()
    if len(years) > 0:
        print(f"\nTemporal Coverage:")
        print(f"  Records with years: {len(years):,}")
        print(f"  Year range: {int(years.min())} - {int(years.max())}")
        print(f"  Span: {int(years.max() - years.min())} years")
        
        # Decade breakdown
        decades = (years // 10) * 10
        decade_counts = decades.value_counts().sort_index()
        if len(decade_counts) > 0:
            print(f"  Most represented decade: {int(decade_counts.idxmax())}s ({decade_counts.max():,} releases)")
    else:
        print(f"\nTemporal Coverage: No year data found")
    
    # Data quality assessment
    print(f"\nData Quality Assessment:")
    missing_data = {}
    key_fields = ['title', 'artist', 'genre', 'year', 'rating']
    has_missing = False
    
    for field in key_fields:
        if field in catalog_df.columns:
            missing_count = catalog_df[field].isnull().sum()
            missing_pct = (missing_count / total_records) * 100
            missing_data[field] = missing_pct
            if missing_pct > 0:
                print(f"  {field.title()}: {missing_pct:.1f}% missing")
                has_missing = True
    
    if not has_missing:
        print("  ✅ No missing data in key fields")
    
    # Memory and storage info
    memory_mb = catalog_df.memory_usage(deep=True).sum() / 1024 / 1024
    print(f"\nStorage Information:")
    print(f"  Memory usage: {memory_mb:.1f} MB")
    print(f"  Average record size: {memory_mb * 1024 / total_records:.1f} KB")

def create_genre_network_analysis(catalog_df, min_genre_count=50):
    """Analyze genre relationships and patterns"""
    print(f"\n🎵 GENRE NETWORK ANALYSIS")
    print("=" * 50)
    
    # Filter genres with sufficient data
    genre_counts = catalog_df['genre'].value_counts()
    major_genres = genre_counts[genre_counts >= min_genre_count].index.tolist()
    
    if len(major_genres) == 0:
        # Lower the threshold if no genres meet the minimum
        min_genre_count = max(10, genre_counts.max() // 4)
        major_genres = genre_counts[genre_counts >= min_genre_count].index.tolist()
    
    print(f"Analyzing {len(major_genres)} major genres (min {min_genre_count} tracks each):")
    
    genre_stats = {}
    for genre in major_genres:
        genre_data = catalog_df[catalog_df['genre'] == genre]
        
        # Calculate statistics
        ratings = pd.to_numeric(genre_data['rating'], errors='coerce').dropna()
        years = pd.to_numeric(genre_data['year'], errors='coerce').dropna()
        
        genre_stats[genre] = {
            'count': len(genre_data),
            'avg_rating': ratings.mean() if len(ratings) > 0 else 0,
            'avg_year': years.mean() if len(years) > 0 else 0,
            'fma_count': len(genre_data[genre_data['source'] == 'fma_data']),
            'discogs_count': len(genre_data[genre_data['source'] == 'discogs_sample'])
        }
    
    # Display results
    print(f"\nGenre Statistics (sorted by track count):")
    for genre in sorted(genre_stats.keys(), key=lambda x: genre_stats[x]['count'], reverse=True):
        stats = genre_stats[genre]
        print(f"  {genre}:")
        print(f"    Tracks: {stats['count']}")
        print(f"    Avg Rating: {stats['avg_rating']:.2f}" if stats['avg_rating'] > 0 else "    Avg Rating: N/A")
        print(f"    Avg Year: {int(stats['avg_year'])}" if stats['avg_year'] > 0 else "    Avg Year: N/A")
        print(f"    Sources: FMA({stats['fma_count']}) + Discogs({stats['discogs_count']})")
        print()

def export_analysis_reports(catalog_df):
    """Export various analysis reports to files (Fixed Version)"""
    print(f"📄 EXPORTING ANALYSIS REPORTS")
    print("=" * 50)
    
    # 1. Genre summary report (fixed)
    try:
        genre_summary = []
        
        for genre in catalog_df['genre'].unique():
            if pd.isna(genre):
                continue
                
            genre_data = catalog_df[catalog_df['genre'] == genre]
            
            # Safe numeric conversions
            ratings = pd.to_numeric(genre_data['rating'], errors='coerce').dropna()
            years = pd.to_numeric(genre_data['year'], errors='coerce').dropna()
            
            genre_info = {
                'genre': genre,
                'track_count': len(genre_data),
                'avg_rating': ratings.mean() if len(ratings) > 0 else None,
                'rating_std': ratings.std() if len(ratings) > 0 else None,
                'avg_year': years.mean() if len(years) > 0 else None,
                'min_year': years.min() if len(years) > 0 else None,
                'max_year': years.max() if len(years) > 0 else None,
                'fma_count': len(genre_data[genre_data['source'] == 'fma_data']),
                'discogs_count': len(genre_data[genre_data['source'] == 'discogs_sample'])
            }
            genre_summary.append(genre_info)
        
        if genre_summary:
            genre_report_df = pd.DataFrame(genre_summary)
            # Round numeric columns
            numeric_cols = ['avg_rating', 'rating_std', 'avg_year', 'min_year', 'max_year']
            for col in numeric_cols:
                if col in genre_report_df.columns:
                    genre_report_df[col] = genre_report_df[col].round(2)
            
            genre_report_path = PROCESSED_DIR / 'genre_analysis_report.csv'
            genre_report_df.to_csv(genre_report_path, index=False)
            print(f"✅ Genre report saved: {genre_report_path}")
        else:
            print("⚠️ No genre data available for report")
        
    except Exception as e:
        print(f"❌ Genre report error: {e}")
    
    # 2. Source comparison report (fixed)
    try:
        source_summary = []
        
        for source in catalog_df['source'].unique():
            if pd.isna(source):
                continue
                
            source_data = catalog_df[catalog_df['source'] == source]
            
            # Safe numeric conversions
            ratings = pd.to_numeric(source_data['rating'], errors='coerce').dropna()
            years = pd.to_numeric(source_data['year'], errors='coerce').dropna()
            
            source_info = {
                'source': source,
                'track_count': len(source_data),
                'avg_rating': ratings.mean() if len(ratings) > 0 else None,
                'rating_std': ratings.std() if len(ratings) > 0 else None,
                'avg_year': years.mean() if len(years) > 0 else None,
                'min_year': years.min() if len(years) > 0 else None,
                'max_year': years.max() if len(years) > 0 else None,
                'unique_genres': source_data['genre'].nunique(),
                'top_genre': source_data['genre'].value_counts().index[0] if len(source_data) > 0 else None
            }
            source_summary.append(source_info)
        
        if source_summary:
            source_report_df = pd.DataFrame(source_summary)
            # Round numeric columns
            numeric_cols = ['avg_rating', 'rating_std', 'avg_year', 'min_year', 'max_year']
            for col in numeric_cols:
                if col in source_report_df.columns:
                    source_report_df[col] = source_report_df[col].round(2)
            
            source_report_path = PROCESSED_DIR / 'source_comparison_report.csv'
            source_report_df.to_csv(source_report_path, index=False)
            print(f"✅ Source comparison saved: {source_report_path}")
        else:
            print("⚠️ No source data available for report")
        
    except Exception as e:
        print(f"❌ Source report error: {e}")
    
    # 3. High-rated tracks report
    try:
        # Convert ratings to numeric and filter
        numeric_ratings = pd.to_numeric(catalog_df['rating'], errors='coerce')
        high_rated_mask = numeric_ratings >= 4.5
        high_rated = catalog_df[high_rated_mask].copy()
        
        if len(high_rated) > 0:
            # Select key columns for the report
            report_columns = ['title', 'artist', 'genre', 'year', 'rating', 'source', 'label']
            available_columns = [col for col in report_columns if col in high_rated.columns]
            
            high_rated_report = high_rated[available_columns].copy()
            
            # Sort by rating descending
            if 'rating' in high_rated_report.columns:
                high_rated_report = high_rated_report.sort_values('rating', ascending=False)
            
            high_rated_path = PROCESSED_DIR / 'high_rated_tracks.csv'
            high_rated_report.to_csv(high_rated_path, index=False)
            print(f"✅ High-rated tracks ({len(high_rated_report)}) saved: {high_rated_path}")
        else:
            print("⚠️ No tracks with rating >= 4.5 found")
        
    except Exception as e:
        print(f"❌ High-rated tracks report error: {e}")
    
    # 4. Summary statistics report
    try:
        summary_stats = {
            'total_records': len(catalog_df),
            'unique_artists': catalog_df['artist'].nunique() if 'artist' in catalog_df.columns else 0,
            'unique_genres': catalog_df['genre'].nunique() if 'genre' in catalog_df.columns else 0,
            'fma_records': len(catalog_df[catalog_df['source'] == 'fma_data']),
            'discogs_records': len(catalog_df[catalog_df['source'] == 'discogs_sample']),
            'records_with_ratings': len(pd.to_numeric(catalog_df['rating'], errors='coerce').dropna()),
            'records_with_years': len(pd.to_numeric(catalog_df['year'], errors='coerce').dropna()),
            'avg_rating': pd.to_numeric(catalog_df['rating'], errors='coerce').mean(),
            'earliest_year': pd.to_numeric(catalog_df['year'], errors='coerce').min(),
            'latest_year': pd.to_numeric(catalog_df['year'], errors='coerce').max()
        }
        
        summary_df = pd.DataFrame([summary_stats])
        summary_df = summary_df.round(2)
        
        summary_path = PROCESSED_DIR / 'catalog_summary_stats.csv'
        summary_df.to_csv(summary_path, index=False)
        print(f"✅ Summary statistics saved: {summary_path}")
        
    except Exception as e:
        print(f"❌ Summary stats report error: {e}")
    
    print(f"\nAll reports saved to: {PROCESSED_DIR}")

# Execute analysis pipeline
print("🚀 Starting comprehensive catalog analysis...")
print()

# 1. Generate statistics
analyze_catalog_statistics(final_catalog)

# 2. Create visualizations
print(f"\n📊 Creating visualizations...")
create_source_comparison_viz(final_catalog)

# 3. Genre network analysis  
create_genre_network_analysis(final_catalog)

# 4. Export reports
export_analysis_reports(final_catalog)

print(f"\n🎉 ANALYSIS COMPLETE!")
print(f"Your enhanced catalog with {len(final_catalog):,} tracks is ready for:")
print(f"  ✅ Advanced searching and filtering")
print(f"  ✅ Cross-source recommendations") 
print(f"  ✅ Genre-based discovery")
print(f"  ✅ Integration with your Streamlit dashboard")
print(f"\nNext steps: Update your vinyl_dashboard.py to use the combined catalog!")
print(f"\nFiles created:")
print(f"  📊 {PROCESSED_DIR / 'combined_catalog_with_discogs.csv'}")
print(f"  📈 {PROCESSED_DIR / 'genre_analysis_report.csv'}")
print(f"  📋 {PROCESSED_DIR / 'source_comparison_report.csv'}")
print(f"  ⭐ {PROCESSED_DIR / 'high_rated_tracks.csv'}")
print(f"  📊 {PROCESSED_DIR / 'catalog_summary_stats.csv'}")